# Análisis de los datos

Se analizan los datos del Corpus PFG entregados.

# Parameters

*  GoldStandardTrainingEAs.txt: contiene una relación de IDS de actos clínicos seguidos de una lista de uno o varios códigos EAs, en caso de que sean aplicables, o NONE en caso de no se haya idetnificado ningún código de efecto adverso. Todos los valores están separados por espacios. Ejemplos:
```
24866017 T50.2X5A
27448436 T83.021A T83.511A T83.091A Y84.6
25205191 NONE
```
*  GoldStandardTestEAs.txt: equivalente al anterior pero con los datos del conjunto de pruebas.
*  EAs.txt: listado de todos los códigos de efectos adversos del CIE10 y su tipo. Ejemplo: 
```
D61.810 - PROC
D61.811 - MED
```
*  CIE-10.tsv: definición del CIE10 completo.
*  Training: carpeta que contiene todos los informes médicos del conjunto de entrenamiento. El nombre del archivo está compuesto por el acto clínico y el identificador del informe. Ejemplo: 23062488-158483734.txt
*  Training: equivalente a training pero con el conjunto de pruebas.



In [1]:
base_path = '/drive/My Drive/CorpusPFG/'

#Datos de origen
GoldStandardTrainingEA_data_location = base_path + 'GoldStandardTrainingEAs.txt'
GoldStandardTestEA_data_location = base_path + 'GoldStandardTestEAs.txt'
EAs_data_location = base_path + 'EAs.txt'
CIE10_data_location = base_path + 'CIE-10.tsv'
training_files_location = base_path + 'Training/'
test_files_location = base_path + 'Test/'

#Punto intermedio en el que se almacenan los datos de los datasets
eas_dataset_training_file = base_path + 'eas-dataset-training.csv'
eas_dataset_test_file = base_path + 'eas-dataset-test.csv'

Se realiza el montaje de la unidad de Google Drive para acceder a los ficheros

In [2]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


Carga del archivo de texto GoldStandardTrainingEAs.txt en un data set

In [3]:
import pandas as pd

column_names = ["Acto", "TieneEA_EA1", "EA2", "EA3", "EA4", "EA5", "EA6", "EA7", "EA8", "EA9", "EA10"]
df_goldStandardTrainingEAs = pd.read_csv(GoldStandardTrainingEA_data_location, sep=' ', header=None, names=[column_names])
df_goldStandardTestEAs = pd.read_csv(GoldStandardTestEA_data_location, sep=' ', header=None, names=[column_names])

TypeError: ignored

Consulta la cantidad de datos



In [ ]:
df_goldStandardTrainingEAs.count()

Consulta de las primeras filas del data set

In [ ]:
df_goldStandardTrainingEAs.head()

In [ ]:
ds_cantidad_TieneEA_EA1 = df_goldStandardTrainingEAs.TieneEA_EA1.value_counts(dropna=True, sort=True)
ds_cantidad_EA2 = df_goldStandardTrainingEAs.EA2.value_counts(dropna=True, sort=True)
ds_all = ds_cantidad_TieneEA_EA1.append(ds_cantidad_EA2)
ds_cantidad_EA3 = df_goldStandardTrainingEAs.EA3.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA3)
ds_cantidad_EA4 = df_goldStandardTrainingEAs.EA4.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA4)
ds_cantidad_EA5 = df_goldStandardTrainingEAs.EA5.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA5)
ds_cantidad_EA6 = df_goldStandardTrainingEAs.EA6.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA6)
ds_cantidad_EA7 = df_goldStandardTrainingEAs.EA7.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA7)
ds_cantidad_EA8 = df_goldStandardTrainingEAs.EA8.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA8)
ds_cantidad_EA9 = df_goldStandardTrainingEAs.EA9.value_counts(dropna=True, sort=True)
ds_all = ds_all.append(ds_cantidad_EA9)

df_cantidad = pd.DataFrame(ds_all)
df_cantidad = df_cantidad.reset_index()
df_cantidad.columns = ['eas', 'counts'] 
df_cantidad.count()

In [ ]:
df_cantidad.plot(x='eas', y='counts', kind='line')	

In [ ]:
df_cantidad_con_EAs = df_cantidad.query("eas != 'NONE'")
df_cantidad_con_EAs.plot(x='eas', y='counts', kind='line')	

**Estadísticas de EAs**

In [ ]:
df_cantidad.query("eas != 'NONE'").value_counts()

In [ ]:
num_actos_clinicos = df_goldStandardTrainingEAs.Acto.count()
num_actos_clinicos_sin_EA = df_goldStandardTrainingEAs.query("TieneEA_EA1 == 'NONE'").TieneEA_EA1.count()
num_actos_clinicos_con_EA = num_actos_clinicos - num_actos_clinicos_sin_EA
num_eas_con_caso_clinico = df_cantidad.query("eas != 'NONE'")["eas"].nunique()

print("Actos clínicos: %s" % num_actos_clinicos)
print("Número de actos clínicos sin EA: %s, un %s%%" % (num_actos_clinicos_sin_EA, round(num_actos_clinicos_sin_EA/num_actos_clinicos*100, 0)))
print("Número de actos clínicos con EA: %s, un %s%%" % (num_actos_clinicos_con_EA, round(num_actos_clinicos_con_EA/num_actos_clinicos*100, 0)))
print("EAs diferentes encontrados en los actos clínicos: %s" % num_eas_con_caso_clinico)
print("")
print("Los 10 códigos CIE más populares:")

df_cantidad.query("eas != 'NONE'").head(10)

#EAs disponibles

Se han identificado 4 tipos:
* *MED*: efectos adversos a medicamentos, vacunas o similar.
* *PROC*: complicaciones en procedimientos (por ej. cirugías).
* *DISP*: fallos en dispositivos médicos.
* *INF*: infecciones, ya sean nosocomiales o durante un tratamiento médico.

Carga del archivo de texto EAs.txt en un data set

In [ ]:
df_EAs = pd.read_csv(EAs_data_location, sep='-', header=None, names=["Code", "Tipo"])
df_EAs['Code'] = df_EAs['Code'].str.strip()

Consulta la cantidad de datos

In [ ]:
df_EAs.count()

Consulta de las primeras filas del data set

In [ ]:
df_EAs.head()

Cantidad de EAs por tipo

In [ ]:
df_EAs.Tipo.value_counts()

Relación entre EAs totales frente a los EAs con ejemplos disponibles

In [ ]:
num_eas_totales = df_EAs["Code"].nunique()
rel_eas = round(num_eas_con_caso_clinico/num_eas_totales*100,2)
print("EAs totales: %s" % num_eas_totales)
print("EAs con representanción en caso clínico: %s" % num_eas_con_caso_clinico)
print("El %s%% de los EAs se dan en al menos un caso clínico." % rel_eas)

#Definición del CIE10

Carga del archivo de texto CIE-10.tsv en un data set

In [ ]:
df_CIE10 = pd.read_csv(CIE10_data_location, sep='\t', header=(0), low_memory=False)
df_CIE10.info()

Consulta de las primeras filas del data set

In [ ]:
df_CIE10.head()

Conteo de los datos del dataset.

In [ ]:
df_CIE10.count()

Tomamos el fichero de códigos EAs y buscamos los registros en el fichero del CIE 10 que coincidan con su código.

In [ ]:
listaCodigosEAs = list(df_EAs.Code.values)
df_CIE10_EAs_Parcial = df_CIE10[df_CIE10['Code'].isin(listaCodigosEAs)]
df_CIE10_EAs_Parcial.head()

Contamos los registros encontrados en el fichero CIE10. El número coincide con el número de dato del fichero EAs.

In [ ]:
df_CIE10_EAs_Parcial.count()

De los datos encontrados en el fichero CIE10, se determinan cuántos de los códigos son finales y los que no lo son.

In [ ]:
df_CIE10_EAs_Parcial.Final.value_counts()

Se crea una lista con los códigos no finales, por cada uno de ellos se buscan los que continúan la jerarquía. Por ejemplo, para el código "D70" que no es final, se buscan los códigos "D70.0", "D70.1", "D70.2", ... que sí lo son. Además, se a la lista de códigos EA completa los códigos finales.

In [ ]:
listaCodigosEAsCompleta=[]
listaCodigosEAsNoFinal = list(df_CIE10_EAs_Parcial[df_CIE10_EAs_Parcial.Final == 0].Code.values)
for ea in listaCodigosEAsNoFinal:
  listaCodigosEAsCompleta.extend(list(df_CIE10[df_CIE10['Code'].str.startswith(ea)].Code))
listaCodigosEAsCompleta.extend(list(df_CIE10_EAs_Parcial[df_CIE10_EAs_Parcial.Final == 1].Code.values))

Se toma la lista de códigos completa y se busca sobre el fichero CIE10, obteniendo con ello el subconjunto de códigos CIE10 que se correspoden con EAs.

In [ ]:
df_CIE10_EAs_Completa = df_CIE10[df_CIE10['Code'].isin(listaCodigosEAsCompleta)]


In [ ]:
df_CIE10_EAs_Completa.count()

In [ ]:
num_eas_totales = df_CIE10_EAs_Completa["Code"].nunique()
rel_eas = round(num_eas_con_caso_clinico/num_eas_totales*100,2)
print("EAs totales (considerando la lista completa): %s" % num_eas_totales)
print("EAs con representanción en caso clínico: %s" % num_eas_con_caso_clinico)
print("El %s%% de los EAs se dan en al menos un caso clínico." % rel_eas)

In [ ]:
#df_CIE10_EAs_Completa.to_csv(base_path + 'df_CIE10_EAs_Completa.csv')

# Detección de datos más prometedores
Obtener las casos clínicos con más EAs más repetidos para crear un dataset 
balanceado y hacer pruebas.

In [ ]:
df_cantidad.query("eas != 'NONE'").head(10)